In [36]:
# K-means
import pandas as pd
import numpy as np
from sklearn import datasets
import random
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

In [37]:
# read iris data and label
iris = datasets.load_iris() # setosa, versicolor, virginica
labels = pd.DataFrame(iris.target)
labels.columns=['labels']

# read iris data and label
data = pd.DataFrame(iris.data)
data.columns=['Sepal length', 'Sepal width','Petal length','Petal width']
data = pd.concat([data,labels],axis=1)
size = len(data)
k = 3
print(data)

     Sepal length  Sepal width  Petal length  Petal width  labels
0             5.1          3.5           1.4          0.2       0
1             4.9          3.0           1.4          0.2       0
2             4.7          3.2           1.3          0.2       0
3             4.6          3.1           1.5          0.2       0
4             5.0          3.6           1.4          0.2       0
5             5.4          3.9           1.7          0.4       0
6             4.6          3.4           1.4          0.3       0
7             5.0          3.4           1.5          0.2       0
8             4.4          2.9           1.4          0.2       0
9             4.9          3.1           1.5          0.1       0
10            5.4          3.7           1.5          0.2       0
11            4.8          3.4           1.6          0.2       0
12            4.8          3.0           1.4          0.1       0
13            4.3          3.0           1.1          0.1       0
14        

In [38]:
# description : calculate distance between two point(pivot and some points in the cluster) using Euclidean method
# p1 : pivot
# p2 : point in the cluster
# return : distance between two points
def euclideanDistance(p1,p2) :
    return np.sqrt(np.square(p2[0]-p1[0])+np.square(p2[1]-p1[1])+np.square(p2[2]-p1[2])+np.square(p2[3]-p1[3]))

# description : calculate accuracy K-means algorithm
# correct : the number of labels which is same with actual label
# size : the number of data
# return : calculated accuracy
def accuracy(correct,size) :
    return correct/size * 100

In [39]:
c = [] 

# Select 3 pivot randomly
index = random.sample(range(150),3)
c.append(data.loc[index[0]].values)
c.append(data.loc[index[1]].values)
c.append(data.loc[index[2]].values)
print(c) # first pivot


distance = [] # Distance Matrix
cluster = [] # cluster. index of points will be added


flag = 1 # Signal, which represent some change in the cluster
        # 0 : not change, 1 : there are some changes.


roundN = 0 # the number of cluster round 

while flag != 0 : # 변한게 없을 때까지 cluster 돌기 # flag = 0 --> 변한게 없다
    flag = 0 # flag 초기화
    distance = [] # 초기화
    
    #1 - Make Distance Matrix 
    for i in range(k) : # pivot
        distance.append([]) # add lists dynamically
        for j in range(size) : # calculate distances about all data
            distance[i].append(euclideanDistance(c[i],data.loc[j]))            
    
    #2 - Enter nearest cluster using distance matrix 
    for i in range(size) :
        # Indentify the nearest pivot
        distanceMin = np.min(np.array([distance[0][i],distance[1][i],distance[2][i]]))
        
        if(len(cluster) == 0) : # when first cycle
            for index in range(k) :
                cluster.append([]) # add cluster lists dynamically
            flag +=1
        elif(len(cluster) != 0) : # not first cycle - Remove data from existing cluster and add it to the cluster
            # If the nearest pivot is changed, mark with the flag
            if((i in cluster[0]) == True) : # When data is in 1th cluster (exisiting)
                cluster[0].remove(i) # remove the data from exisiting cluster
                if(distanceMin != distance[0][i]) : # the nearest pivot is changed from 0 to other pivot
                    flag +=1 
            elif((i in cluster[1]) == True) : # When data is in 2th cluster (exisiting)
                cluster[1].remove(i) 
                if(distanceMin != distance[1][i]) : 
                    flag += 1
            elif((i in cluster[2]) == True) : # When data is in 3th cluster (exisiting)
                cluster[2].remove(i)
                if(distanceMin != distance[2][i]) : 
                    flag +=1
                 
        # add index of data in the nearest cluster
        if (distanceMin == distance[0][i]) : cluster[0].append(i)
        elif (distanceMin == distance[1][i]) : cluster[1].append(i)
        elif (distanceMin == distance[2][i]) : cluster[2].append(i)

    c=[] # Initialize
    
    #3 - Find new pivot of clusters
    for i in range(k) :
        c.append( np.array([ np.mean(data.iloc[cluster[i],0]),
                         np.mean(data.iloc[cluster[i],1]),
                         np.mean(data.iloc[cluster[i],2]),
                         np.mean(data.iloc[cluster[i],3]) ]) )

cluster_result = []

for i in range(k) : # each cluster
    print("* ",i,"th cluster *\n")
    print(data.loc[cluster[i]])
    result = Counter(data.iloc[cluster[i],4])
    cluster_result.append(result.most_common(1)[0][0])
    print("\n\n")
    
cluster_label = []
correct = 0;
for i in range(size) :
    if (i in cluster[0]) == True : cluster_label.append(cluster_result[0])
    elif (i in cluster[1]) == True : cluster_label.append(cluster_result[1])
    elif (i in cluster[2]) == True : cluster_label.append(cluster_result[2])
    
    if (cluster_label[i] == data.iloc[i,4]) :
        correct +=1
        

print(cluster_result) # Labels represented by each cluster
print("correct: ",correct) # the number of label which is same with actual label
print("Accuracy: ",accuracy(correct,size)) # the number of label which is same with actual label / size of all data
print(cluster_label ) # Labels determined by clusters

[array([5.7, 2.8, 4.5, 1.3, 1. ]), array([5.6, 2.5, 3.9, 1.1, 1. ]), array([5.6, 2.9, 3.6, 1.3, 1. ])]
*  0 th cluster *

     Sepal length  Sepal width  Petal length  Petal width  labels
50            7.0          3.2           4.7          1.4       1
52            6.9          3.1           4.9          1.5       1
77            6.7          3.0           5.0          1.7       1
100           6.3          3.3           6.0          2.5       2
102           7.1          3.0           5.9          2.1       2
103           6.3          2.9           5.6          1.8       2
104           6.5          3.0           5.8          2.2       2
105           7.6          3.0           6.6          2.1       2
107           7.3          2.9           6.3          1.8       2
108           6.7          2.5           5.8          1.8       2
109           7.2          3.6           6.1          2.5       2
110           6.5          3.2           5.1          2.0       2
111           6.4   